In [2]:
# !pip3 install google-cloud-aiplatform --upgrade --user

In [3]:
from absl import app
from absl import flags
from absl import logging
from google.cloud import aiplatform as vertex_ai
import os
import time

In [7]:
PREFIX = 'twotower' 
PREFIX = 'lowes-rec'
DISPLAY_NAME = f'{PREFIX}-tensorboard'
PROJECT= 'lowes-reccomendation'
REGION='us-central1'
STAGING_BUCKET = 'gs://tfrs-central-a/models'
TENSORBOARD = 'projects/258043323883/locations/us-central1/tensorboards/1819067221364703232' # TODO args
VERTEX_SA = '258043323883-compute@developer.gserviceaccount.com'

In [8]:
# !gcloud beta ai tensorboards create --display-name $DISPLAY_NAME --project $PROJECT --region $REGION

In [9]:
# initialize vertex sdk
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

In [5]:
# flags.DEFINE_string('bucket_name', None, 'GCS bucket name without gs://')
# flags.DEFINE_string('filename_pattern', None, 'e.g., gs://{BUCKET_NAME}/*.tfrec')
# flags.DEFINE_string('tb_instance', None, 'tensorboard instance')
# flags.DEFINE_integer('embed_dim', 32, 'embedding dimension')
# flags.DEFINE_integer('max_tokens', 1000000, 'max tokens for string vocab')
# flags.DEFINE_integer('batch_size', 128, 'per replica batch size')
# flags.DEFINE_integer('epochs', 5, 'training epochs')

In [10]:
!mkdir vertex-train

mkdir: cannot create directory ‘vertex-train’: File exists


In [15]:
%%writefile vertex-train/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

RUN pip install -r trainer/requirements.txt

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Overwriting vertex-train/Dockerfile


In [16]:
%%writefile vertex-train/trainer/requirements.txt

tensorflow-recommenders==0.5.2
tensorflow-io-gcs-filesystem==0.18.0
absl-py==0.12.0
google-api-python-client==1.12.8
tfx-bsl==1.1.1
google-cloud-bigquery==2.20.0
dill==0.3.1.1
pyarrow==2.0.0
tfx-bsl==1.1.1
tensorflow-transform==1.1.0

Overwriting vertex-train/trainer/requirements.txt


In [19]:
%%writefile vertex-train/trainer/task.py

import json
import os
from classes_functions import QueryModel, TheTwoTowers, ProductModel, parse_tfrecord_fn, parse_tfrecord_catalog
from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_recommenders as tfrs
from google.cloud import storage


FLAGS = flags.FLAGS

def _get_model_dir(model_dir):
    """Defines utility functions for model saving.

    In a multi-worker scenario, the chief worker will save to the
    desired model directory, while the other workers will save the model to
    temporary directories. It’s important that these temporary directories
    are unique in order to prevent multiple workers from writing to the same
    location. Saving can contain collective ops, so all workers must save and
    not just the chief.
    """

    def _is_chief(task_type, task_id):
        return ((task_type == 'chief' and task_id == 0) or task_type is None)

    tf_config = os.getenv('TF_CONFIG')
    if tf_config:
        tf_config = json.loads(tf_config)

    if not _is_chief(tf_config['task']['type'], tf_config['task']['index']):
        model_dir = os.path.join(model_dir,
                                 'worker-{}').format(tf_config['task']['index'])

    logging.info('Setting model_dir to: %s', model_dir)

    return model_dir

def main(_):
    
    ## set params TODO - make params work with `absl`
    
    EMBEDDING_DIM = 32
    MAX_TOKENS = 50_000 #1265634
    N_PRODUCTS = 10_000 #212_862 212861
    N_HEIR = 10_000 #54724 
    BATCH_SIZE = 2048
    ARCH = [256, 128, 64]
    NUM_EPOCHS = 30
    TF_RECORDS_DIR = 'gs://tfrs-central-a'
    TF_RECORDS_CAT = 'gs://prod-catalog-central'
    NUM_RECORDS = 4_293_302
    
    # Sets mixed_precision policy. Using 'mixed_float16' or 'mixed_bfloat16'
  # can have significant impact on model speeds by utilizing float16 in case of
  # GPUs, and bfloat16 in the case of TPUs. loss_scale takes effect only when
  # dtype is float16
    if params.runtime.mixed_precision_dtype:
        performance.set_mixed_precision_policy(params.runtime.mixed_precision_dtype)
    distribution_strategy = distribute_utils.get_distribution_strategy(
        distribution_strategy=params.runtime.distribution_strategy,
        all_reduce_alg=params.runtime.all_reduce_alg,
        num_gpus=params.runtime.num_gpus,
        tpu_address=params.runtime.tpu,
        **params.runtime.model_parallelism())

    
    ## get product, query data files
    client = storage.Client()
    files = []
    for blob in client.list_blobs('tfrs-tf-records'):
        files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

    
    files_cat = []
    for blob in client.list_blobs('prod-catalog-central'):
        files_cat.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

    print(files_cat[:2])client = storage.Client()
    files = []
    for blob in client.list_blobs('tfrs-tf-records'):
        files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))


    files_cat = []
    for blob in client.list_blobs('prod-catalog-central'):
        files_cat.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

    ## establish the pipelines
    # Set dev dataset CHANGE THIS LATER TO THE WHOLE DIR
    raw_dataset = tf.data.TFRecordDataset(files) #local machine training wheels - using smaller data set for starters
    cat_dataset = tf.data.TFRecordDataset(files_cat)


    #See `pipeline-opts.ipynb` for more info on tuning options
    parsed_dataset = raw_dataset.map(
            parse_tfrecord_fn,
            num_parallel_calls=tf.data.AUTOTUNE
        ).prefetch(  # Overlap producer and consumer works
            tf.data.AUTOTUNE
        )


    # Doing another pipeline for the adapts to get startup to run much faster

    parsed_dataset_adapt = raw_dataset.map(
        parse_tfrecord_fn,
        num_parallel_calls=tf.data.AUTOTUNE
        ).prefetch(  # Overlap producer and consumer works
            tf.data.AUTOTUNE
        )

    parsed_dataset_adapt = parsed_dataset_adapt.batch(BATCH_SIZE)

    # parsed_dataset_adapt = parsed_dataset_adapt.batch(BATCH_SIZE)
    # loading de-duplicated product catalog

    parsed_dataset_candidates = cat_dataset.map(
            parse_tfrecord_catalog,
            num_parallel_calls=tf.data.AUTOTUNE
        ).prefetch(  # Overlap producer and consumer works
            tf.data.AUTOTUNE
        )

    parsed_dataset_candidates = parsed_dataset_candidates.cache()
    tf.random.set_seed(42)

    shuffled = parsed_dataset.shuffle(200_000, seed=42, reshuffle_each_iteration=False)
    # shuffled = shuffled.cache()

    test_pct = 0.05
    n_test = int(NUM_RECORDS * test_pct)

    # train_records
    test = shuffled.take(n_test)
    train = shuffled.skip(n_test)

    cached_train = train.batch(BATCH_SIZE)
    cached_test = test.batch(BATCH_SIZE * 2).cache()
    
    model = TheTwoTowers(ARCH)

    model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
    
    layer_history = model.fit(
        cached_train,
        validation_data=cached_test,
        validation_freq=5,
        epochs=NUM_EPOCHS,
        verbose=0)

    accuracy = layer_history.history["factorized_top_k/top_100_categorical_accuracy"][-1]
    print(f"Top-100 accuracy: {accuracy:.2f}.")

Overwriting vertex-train/trainer/task.py


## Note we manually copy/pasted the model classes and data parsing functions to `./vertex-train/trainer/classes_functions.py`

In [20]:
%%bash
PROJECT_ID='lowes-reccomendation'
IMAGE_URI="gcr.io/$PROJECT_ID/multiworker:two-tower-jw"
docker build ./vertex-train/ -t $IMAGE_URI
docker push $IMAGE_URI

Sending build context to Docker daemon  87.04kB
Step 1/5 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
 ---> 307b41b1aec7
Step 2/5 : WORKDIR /
 ---> Using cache
 ---> 8b66c51b024f
Step 3/5 : COPY trainer /trainer
 ---> 5e713d678733
Step 4/5 : RUN pip install -r trainer/requirements.txt
 ---> Running in b5ef73c46a9f
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78532 sha256=3050a829011e7444878a2ed8675ac0ec885c6e97a380e554c41c743907cba9ec
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
Successfully built dill
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
  Attempting uninstall: grpcio
    Found exist

## Submit Job from custom container

In [21]:
# TRAIN_IMAGE='gcr.io/lowes-reccomendation/multiworker:two-tower'
TRAIN_IMAGE='gcr.io/lowes-reccomendation/multiworker:two-tower-jw'

In [29]:
# REMINDER ON PARAMS
#     EMBEDDING_DIM = 32
#     MAX_TOKENS = 50_000 #1265634
#     N_PRODUCTS = 10_000 #212_862 212861
#     N_HEIR = 10_000 #54724 
#     BATCH_SIZE = 2048
#     ARCH = [256, 128, 64]
#     NUM_EPOCHS = 30
#     TF_RECORDS_DIR = 'gs://tfrs-central-a'
#     TF_RECORDS_CAT = 'gs://prod-catalog-central'
#     NUM_RECORDS = 4_293_302

worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": "c2-standard-30",
            "accelerator_type": "NVIDIA_TESLA_V100",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_IMAGE,
#            "command": ["python", "train.py"],
            "args": [
                '--bucket_name=tfrs-central-a', 
                '--filename_pattern=gs://{BUCKET_NAME}/*.tfrec',
                '--tb_instance=projects/258043323883/locations/us-central1/tensorboards/1819067221364703232',
                '--epochs=5'
            ],
        },
    }
]

In [30]:
job_name = f'{PREFIX}_CUSTOM_CONTAINER'
base_output_dir = '{}/jobs/{}/{}'.format(STAGING_BUCKET, job_name, time.strftime("%Y%m%d_%H%M%S"))

job = vertex_ai.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=base_output_dir
)

job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD
)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob


## Submit jobs from custom container - v2
[distributed GPU training example](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/reduction_server/distributed-training-reduction-server.ipynb)

Try:
* Worker Pool 0: (1) `n1-highcpu-16`, (2) `v100`
* Worker pool 1: (1) `n1-highcpu-16`, (2) `v100`
* Worker Pool 2: (2) `n1-highcpu-16`

In [31]:
def prepare_worker_pool_specs(
    image_uri,
    args,
    cmd,
    replica_count=1,
    machine_type="n1-standard-4",
    accelerator_count=0,
    accelerator_type="ACCELERATOR_TYPE_UNSPECIFIED",
    reduction_server_count=0,
    reduction_server_machine_type="n1-highcpu-16",
    reduction_server_image_uri="us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest",
):

    if accelerator_count > 0:
        machine_spec = {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        }
    else:
        machine_spec = {"machine_type": machine_type}

    container_spec = {
        "image_uri": image_uri,
        "args": args,
        "command": cmd,
    }

    chief_spec = {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec,
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            "replica_count": replica_count - 1,
            "machine_spec": machine_spec,
            "container_spec": container_spec,
        }
        worker_pool_specs.append(workers_spec)

    if reduction_server_count > 1:
        workers_spec = {
            "replica_count": reduction_server_count,
            "machine_spec": {
                "machine_type": reduction_server_machine_type,
            },
            "container_spec": {"image_uri": reduction_server_image_uri},
        }
        worker_pool_specs.append(workers_spec)

    return worker_pool_specs

In [22]:
job_name = f'{PREFIX}_CUSTOM_SCRIPT'
base_output_dir = '{}/jobs/{}/{}'.format(STAGING_BUCKET, job_name, time.strftime("%Y%m%d_%H%M%S"))

#container_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-4:latest'
container_uri = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-6'
requirements = ['tensorflow-recommenders','tensorboard_plugin_profile']

WORKER_CMD = ["python", "-m", "trainer.task"]
WORKER_ARGS = [
    '--bucket_name=tfrs-central-a', 
    '--filename_pattern=gs://{BUCKET_NAME}/*.tfrec',
    '--tb_instance=projects/258043323883/locations/us-central1/tensorboards/1819067221364703232',
    '--epochs=5',]

REPLICA_COUNT = 1
WORKER_MACHINE_TYPE = "a2-highgpu-4g"
ACCELERATOR_TYPE = "NVIDIA_TESLA_A100"
PER_MACHINE_ACCELERATOR_COUNT = 4
PER_REPLICA_BATCH_SIZE = 32

REDUCTION_SERVER_COUNT = 4
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"


machine_type = 'c2-standard-30' # 'n1-standard-4'
accelerator_type = 'NVIDIA_TESLA_V100' # NVIDIA_TESLA_T4'
accelerator_count = 1
replicas=2

In [ ]:
job_name = f'{PREFIX}_CUSTOM_SCRIPT'
base_output_dir = '{}/jobs/{}/{}'.format(STAGING_BUCKET, job_name, time.strftime("%Y%m%d_%H%M%S"))

#container_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-4:latest'
container_uri = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-5'
requirements = ['tensorflow-recommenders==0.5.2','tensorflow-io-gcs-filesystem==0.18.0', 'absl-py==0.12.0', 'google-api-python-client==1.12.8', 'tfx-bsl==1.1.1', 'google-cloud-bigquery==2.20.0', 'dill==0.3.1.1', 'pyarrow==2.0.0', 'tfx-bsl==1.1.1', 'tensorflow-transform==1.1.0']
args = [
    '--bucket_name=tfrs-central-a', 
    '--filename_pattern=gs://{BUCKET_NAME}/*.tfrec',
    '--tb_instance=projects/258043323883/locations/us-central1/tensorboards/1819067221364703232',
    '--epochs=5',
]

machine_type = 'c2-standard-30' # 'n1-standard-4'
accelerator_type = 'NVIDIA_TESLA_V100' # NVIDIA_TESLA_T4'
accelerator_count = 1
replicas=2

job = vertex_ai.CustomJob.from_local_script(
    display_name=job_name,
    machine_type=machine_type,
    replica_count=replicas,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    script_path='vertex-train/trainer/task.py', # 'trainer/train.py', 
    container_uri=container_uri,
    requirements=requirements,
    args=args,
    staging_bucket=base_output_dir
)

job.run(sync=False, 
        service_account=VERTEX_SA,
        tensorboard=TENSORBOARD,
        )